In [2]:
from sqlalchemy import create_engine
import pymysql
import pandas as pd
import csv
import os
import time


In [3]:
# mysql connect하기 위한 아이디 비밀번호 포트 데이터베이스 등록 및 conn 리턴
def sqlalchemy_connect():
    engine = create_engine("mysql+pymysql://root:"
                +"big15"
                +"@localhost:3306/big15?charset=utf8"
                , encoding='utf8')
    
    return engine.connect()

In [4]:
# mysql connect하기 위한 아이디 비밀번호 포트 데이터베이스 등록 및 conn 리턴
def sqlalchemy_connect_ip(ip_address, db_name):
    engine = create_engine("mysql+pymysql://admin:"
                +"big15" # user password
                +"@{0}:3306/{1}?charset=utf8".format(ip_address, db_name)
                , encoding='utf8')
    
    return engine.connect()

In [12]:
# Data csv => SQL에 추가
def sqlalchemy_data_insert(conn):
    
    # for문 돌려서 파일 리스트 받고 하나씩 읽는다
    for file in os.listdir('../data/update/20221116/concat/'):
        stock_code_name = file.split('_')[-2] + '_' + file.split('_')[-1].split('.')[0]

        stock_info = pd.read_csv('../data/update/20221116/concat/{0}'.format(file))

        stock_info['년'] = stock_info['날짜'].apply(str).str[:4]
        stock_info['월'] = stock_info['날짜'].apply(str).str[4:6]
        stock_info['일'] = stock_info['날짜'].apply(str).str[6:8]
        stock_info.sort_index(ascending=False, inplace=True)
        # name= table명
        # con= connect할 때 들어가는 아이디 비밀번호 등등
        # if_exists= 'append' : 기존 테이블이 있는 경우 데이터를 추가
        # 'fail' : 기존 테이블이 있는 경우 아무일도 없지만 없을 경우 valuesError뜸
        # 'replace' : 기존 테이블이 있을 경우, 기존 테이블을 삭제하고 다시 테이블을
        count = stock_info.to_sql(name='{0}'.format(stock_code_name), con=conn, if_exists='replace', index=False)
        
        stock_info = pd.DataFrame()

        print(count)

    # for문 완료 후 연결 해제
    conn.close()


In [ ]:
sqlalchemy_data_insert(sqlalchemy_connect_ip('192.168.50.123', 'stock_info'))

In [14]:
from module.setting import instCpCodeMgr, instStockChart, instCpCybos
from pykrx import stock
from datetime import datetime
from datetime import timedelta
from random import uniform
from tqdm import tqdm
import pymysql
import csv
import os
import time
import numpy as np

def get_pymysql_connection(ip_address, db_name):

    conn = pymysql.connect(host=ip_address, user='admin', password='big15'
                        , db=db_name, charset='utf8')

    return conn
    
def get_pymysql_stock_list(conn):

    # 현재 DB 내 존재하는 테이블(종목) 추출
    sql = "SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = 'stock_info'"

    with conn:
        with conn.cursor() as cur:
            cur.execute(sql)
            result = [item[0] for item in cur.fetchall()]
            cur.close()

            return result



def get_compare_list(stock_list):
# stock code가 테이블이 존재하는 비교
    today = str(datetime.today().year) + str(datetime.today().month) + str((datetime.today() - timedelta(1)).day)

    kospi = pd.read_csv('../data/kospi_{today}.csv', encoding='euc-kr')
    kosdaq = pd.read_csv('../data/kosdaq_{today}.csv', encoding='euc-kr')

    kospi = kospi[kospi['주식종류'] == '보통주'].iloc[:,[1,-3]].iloc[:,0]
    kosdaq = kosdaq[kosdaq['주식종류'] == '보통주'].iloc[:,[1,-3]].iloc[:,0]

    kospi_kosdaq_list = pd.concat([kospi,kosdaq]).to_list()

    exist_list = []
    empty_list = []

    for code in kospi_kosdaq_list:
        if code in stock_list:
            exist_list.append(code)
        elif code not in stock_list:
            empty_list.append(code)
    
    return exist_list, empty_list

def update_pymysql_exist(code, type, conn):
# 존재할 경우, 대신증권에서 해당 종목 일봉 분봉 업데이트문

    sql = "SELECT MAX(날짜) FROM STOCK_INFO.{0} ".format(code)

    cur = conn.cursor()
    cur.execute(sql)
    result = cur.fetchone() # 마지막 업데이트 날짜
    
    day_format = '%Y%m%d'
    start_day = datetime.strftime(datetime.strptime(result, day_format) + timedelta(days=1), day_format)

    today = str(datetime.today().year) + str(datetime.today().month) + str((datetime.today()).day)

    # 최종 업데이트 날짜가 오늘 날짜와 같은 경우 배제
    if str(result) != today:
        temp_df = pd.DataFrame()
        while True:
            instStockChart.SetInputValue(0, 'A'+code) # 종목명
            instStockChart.SetInputValue(1, ord('1')) # 1 : 기간으로 요청, 2: 개수로 요청
            instStockChart.SetInputValue(3, start_day) # 요청 시작일 22일
            instStockChart.SetInputValue(2, start_day) # 요청 종료일 22일
            instStockChart.SetInputValue(5, [0, 1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17,18, 19,20,21,22,23,24,25,26])
            instStockChart.SetInputValue(6, ord(type)) # 'D' : 일봉, 'm' : 분봉
            instStockChart.SetInputValue(9, ord('1'))

            instStockChart.BlockRequest() # 위 정보로 요청

            numrow, numcolumn = instStockChart.GetHeaderValue(3), instStockChart.GetHeaderValue(2)


            index = []
            for i in range(numrow):
                index_ = str(instStockChart.GetDataValue(0,i))
                index.append(index_)

            
            stock_info = pd.DataFrame(columns=numcolumn[1:], index=index)

            for num in range(numrow):
                for col in range(len(numcolumn)):
                    # 1,2,3,4,5,6,7,8,9, 10
                    stock_info.iloc[num, col-1] = str(instStockChart.GetDataValue(col,num))
                
            temp_df = pd.concat([stock_info, temp_df])
           

            start_day = datetime.strftime(datetime.strptime(start_day, day_format) + timedelta(days=1), day_format)
            transfer_end_day = datetime.strftime(datetime.strptime(today, day_format), day_format)
            time.sleep(uniform(0.15, 0.3))

            if instCpCybos.GetLimitRemainCount(1) < 3:
                time.sleep(10)

            if start_day > transfer_end_day:
                temp_df = temp_df.reset_index().rename(columns={'index':'날짜'})
                return temp_df

def update_pymysql_empty(code, type, conn):
# 존재할 경우, 대신증권에서 해당 종목 일봉 분봉 업데이트문
    
    day_format = '%Y%m%d'
    minus_day = timedelta(days=1)

    temp_df = pd.DataFrame()

    today = str(datetime.today().year) + str(datetime.today().month) + str((datetime.today()).day)

    for day in range(730):
        instStockChart.SetInputValue(0, 'A'+code) # 종목명
        instStockChart.SetInputValue(1, ord('1')) # 1 : 기간으로 요청, 2: 개수로 요청
        instStockChart.SetInputValue(3, today) # 요청 시작일 22일
        instStockChart.SetInputValue(2, today) # 요청 종료일 22일
        instStockChart.SetInputValue(5, [0, 1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17,18, 19,20,21,22,23,24,25,26])
        instStockChart.SetInputValue(6, ord(type)) # 'D' : 일봉, 'm' : 분봉
        instStockChart.SetInputValue(9, ord('1'))

        instStockChart.BlockRequest() # 위 정보로 요청

        numrow, numcolumn = instStockChart.GetHeaderValue(3), instStockChart.GetHeaderValue(2)


        index = []
        for i in range(numrow):
            index_ = str(instStockChart.GetDataValue(0,i))
            index.append(index_)

    
        stock_info = pd.DataFrame(columns=numcolumn[1:], index=index)

        for num in range(numrow):
            for col in range(len(numcolumn)):
                # 1,2,3,4,5,6,7,8,9, 10
                stock_info.iloc[num, col-1] = str(instStockChart.GetDataValue(col,num))
            
        temp_df = pd.concat([stock_info, temp_df])
        
        today = datetime.strftime(datetime.strptime(today, day_format) - timedelta(days=1), day_format)
        transfer_end_day = datetime.strftime(datetime.strptime(today, day_format), day_format)
        time.sleep(uniform(0.15, 0.3))

        if instCpCybos.GetLimitRemainCount(1) < 3:
            time.sleep(10)
            
        temp_df = temp_df.reset_index().rename(columns={'index':'날짜'})
        return temp_df

# 삼성전자 종목 업데이트 우선 수행 후 template 생성

def save_samsung_template():

    today = str(datetime.today().year) + str(datetime.today().month) + str((datetime.today() - timedelta(1)).day)
    yesterday = str(datetime.today().year) + str(datetime.today().month) + str((datetime.today() - timedelta(2)).day)
    template = update_pymysql_exist('A005930', 'm')
    template = template.set_index(['날짜', '시간'])
    template = template.notnull().replace(True, np.NaN)
    y_template = pd.read_csv('../data/{}_fillrow_template.csv'.format(yesterday), encoding='utf-8 sig')
    y_template = y_template.rename(colums={'Unnamed: 0' : '날짜'})
    template = pd.concat([template, y_template], axis=0, ignore_index=True)
    template.to_csv('../data/{}_fillrow_template.csv'.format(today), encoding='utf-8 sig')

    return template


# 종목별 업데이트, fillrow, concat 수행

def update_fillrow(min_df, template=save_samsung_template()):

    min_df = pd.merge(template, min_df, how='left', left_index=True, right_index=True)

    min_df = min_df.drop(columns=min_df.columns[0:23], axis=1)

    columns_temp = min_df.columns

    bin_columns = []
    for column in columns_temp:
        bin_columns.append(column.split('_')[0])

    min_df.columns = bin_columns

    min_df = min_df.reset_index()


    min_df[['거래량','거래대금']] = min_df[['거래량','거래대금']].fillna(0)
    min_df = min_df.bfill()

    return min_df

# 분봉 일봉 concate 진행

def update_concat(min_df, day_df):

    min_drop_list = ['전일대비','상장주식수','시가총액','외국인주문한도수량'
                ,'외국인주문가능수량','외국인현보유수량','외국인현보유비율'
                ,'수정주가일자','수정주가비율','기관순매수량','기관누적순매수량'
                ,'등락주선','등락비율','예탁금','주식회전율','거래성립률']                                  
    day_drop_list = ['시간','시가','고가','저가','종가','거래량','거래대금','누적체결매도수량', '누적체결매수수량'
                ,'등락주선','등락비율','예탁금','주식회전율','거래성립률']
        
    stock_min_df = min_df.drop(min_drop_list, axis=1)
    stock_day_df = day_df.drop(day_drop_list, axis=1)
    
    concat_df = pd.merge(stock_min_df, stock_day_df, on='날짜', how='left')

    return concat_df

# label 추가

def update_label(concat_df):
    
    # 데이터 프레임에서 날짜 인덱스 추출
    date = concat_df.index.unique()
    # label 컬럼 추가 후 0으로 초기화
    concat_df['label'] = 0
    # 업데이트에 사용할 데이터 프레임 생성
    update_stock_info = pd.DataFrame()
    # labeling
    for day in date:
        # 특정일의 Data 추출
        select_day = concat_df.loc[day].copy()
        select_day['label'] = 0
        
        # 특정일의 Row 만큼 반복
        for row in range(len(select_day)):
            # 특정일의 현재 row 이후 최대 고가를 추출
            next_price = select_day[-row-1::-1]['고가'].max()
            # 추출한 최대 고가를 label 컬럼에 대입
            select_day.iloc[-row-1,-1] = next_price
            next_price = 0
        # 특정일 label이 추가된 DF를 업데이트할 DF에 concat
        update_stock_info = pd.concat([update_stock_info, select_day])
        
    return update_stock_info


# Data csv => SQL에 추가
def sqlalchemy_update_insert(update_df, code, type, conn=conn):

    update_df['년'] = update_df['날짜'].apply(str).str[:4]
    update_df['월'] = update_df['날짜'].apply(str).str[4:6]
    update_df['일'] = update_df['날짜'].apply(str).str[6:8]
    update_df.sort_index(ascending=False, inplace=True)
    # name= table명
    # con= connect할 때 들어가는 아이디 비밀번호 등등
    # if_exists= 'append' : 기존 테이블이 있는 경우 데이터를 추가
    # 'fail' : 기존 테이블이 있는 경우 아무일도 없지만 없을 경우 valuesError뜸
    # 'replace' : 기존 테이블이 있을 경우, 기존 테이블을 삭제하고 다시 테이블을
    count = update_df.to_sql(name='{0}'.format(code), con=conn, if_exists=type, index=False)
    
# for문 완료 후 연결 해제
    conn.close()




##########################################################################################################

conn = get_pymysql_connection('192.168.50.123', 'stock_info')
stock_list = get_pymysql_stock_list(conn)
exist_list, empty_list = get_compare_list(stock_list)

for code in exist_list:

    min_df = update_pymysql_exist(code, 'm', conn)
    day_df = update_pymysql_exist(code, 'D', conn)

    min_df = update_fillrow(min_df)
    concat_df = update_concat(min_df, day_df)
    update_df = update_label(concat_df)
    sqlalchemy_update_insert(update_df, code, 'append')


for code in empty_list:

    min_df = update_pymysql_exist(code, 'm', conn)
    day_df = update_pymysql_exist(code, 'D', conn)

    min_df = update_fillrow(min_df)
    concat_df = update_concat(min_df, day_df)
    update_df = update_label(concat_df)
    sqlalchemy_update_insert(update_df, code, 'replace')
    





















# 존재하지 않는 경우, 새로운 테이블을 만들고 해당 종목 추가


# curs = conn.cursor()
# print(curs)
# print(curs.execute(sql))
# print(sql)
# curs.close()

# conn.commit()

# sql = "insert into user (id, name, region, insdt) values (%s, %s, %s, now())"
# f = open('test.csv', 'r', encoding='utf-8')
# rd = csv.reader(f)

# for line in rd:
#     curs.execute(sql, (line[0], line[1], line[2]))

# conn.commit()
# conn.close()
# f.close()







IndentationError: unexpected indent (422747130.py, line 57)

In [18]:
test = pd.read_csv('../data/fillrow_template.csv')
test

,Unnamed: 0,시간,시가,고가,저가,종가,전일대비,거래량,거래대금,누적체결매도수량,...,외국인현보유비율,수정주가일자,수정주가비율,기관순매수량,기관누적순매수량,등락주선,등락비율,예탁금,주식회전율,거래성립률
0,20221121,1530,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20221121,1520,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20221121,1519,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20221121,1518,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20221121,1517,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194566,20201028,905,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
194567,20201028,904,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
194568,20201028,903,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
194569,20201028,902,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
print(result)

['daily_price']


In [6]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from datetime import datetime
import glob
import shutil
import time

target = r"D:/systrader-dev/data/kos*.csv"
csv_list = glob.glob(target)

if len(csv_list) >= 1:
    os.remove(csv_list[0])
    os.remove(csv_list[1])

driver = webdriver.Chrome("./chromedriver")

driver.get("http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201020201")
driver.implicitly_wait(20)
time.sleep(4)
driver.find_element(By.XPATH, '//*[@id="MDCSTAT019_FORM"]/div[1]/div/table/tbody/tr/td/label[2]').click()
time.sleep(1.5)
driver.implicitly_wait(20)
driver.find_element(By.XPATH, '//*[@id="MDCSTAT019_FORM"]/div[2]/div[1]/p[2]/button[2]').click()	
time.sleep(1.5)
driver.implicitly_wait(20)
driver.find_element(By.XPATH, '//*[@id="ui-id-1"]/div/div[2]').click()
time.sleep(1.5)
driver.implicitly_wait(20)
driver.find_element(By.XPATH, '//*[@id="MDCSTAT019_FORM"]/div[1]/div/table/tbody/tr/td/label[3]').click()
time.sleep(1.5)
driver.implicitly_wait(20)
driver.find_element(By.XPATH, '//*[@id="MDCSTAT019_FORM"]/div[2]/div[1]/p[2]/button[2]').click()	
time.sleep(1.5)
driver.implicitly_wait(20)
driver.find_element(By.XPATH, '//*[@id="ui-id-3"]/div/div[2]').click()
time.sleep(1.5)
driver.close()

target = r"C:Users/TJ/Downloads/data*.csv"
new_csv_list = glob.glob(target)

file_size_1 = os.path.getsize(new_csv_list[0])
file_size_2 = os.path.getsize(new_csv_list[1])

now = datetime.now()
cur_time = now.strftime('%Y%m%d')

if file_size_1 < file_size_2:
    shutil.move(new_csv_list[0], "../data/kospi_{0}.csv".format(cur_time))
    shutil.move(new_csv_list[1], "../data/kosdaq_{0}.csv".format(cur_time))
else:
    shutil.move(new_csv_list[1], "../data/kospi_{0}.csv".format(cur_time))
    shutil.move(new_csv_list[0], "../data/kosdaq_{0}.csv".format(cur_time))



C:\Users\TJ\AppData\Local\Temp\ipykernel_7408\2883045696.py:16: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("./chromedriver")


In [ ]:
host_name = "localhost"
username = 'root'
password = 'big15'
database_name = 'big15'

In [ ]:
db1 = pymysql.connect(
    host= host_name
    , port=3306
    , user=username
    , password=password
    , db=database_name
    , charset='utf8'
)

In [ ]:
sql = 'select * from stock'
df2 = pd.read_sql(sql,con=db1)
print(df2)

c:\Users\TJ\.conda\envs\tf2\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


        Unnamed: 0    시간     시가     고가     저가     종가  전일대비     거래량  \
0         20221103  1530  82700  82700  82700  82700     0  312503   
1         20221103  1520  83100  83100  83000  83100     0   12246   
2         20221103  1519  83000  83100  83000  83100     0   19656   
3         20221103  1518  83200  83200  83000  83000     0   39559   
4         20221103  1517  83100  83200  83100  83200     0    4789   
...            ...   ...    ...    ...    ...    ...   ...     ...   
190375    20201027   905  82000  82300  81800  82200     0   53979   
190376    20201027   904  82200  82300  82000  82000     0   19926   
190377    20201027   903  82200  82300  82100  82200     0   16356   
190378    20201027   902  82500  82500  82200  82200     0   12931   
190379    20201027   901  82300  82600  82200  82600     0   41258   

               거래대금  누적체결매도수량  ...  외국인현보유비율  수정주가일자  수정주가비율  기관순매수량  \
0       25843990000   1314512  ...       0.0       0     0.0       0   
1        101726

In [ ]:
df2 = df2.astype({'Unnamed: 0':'int32'})
df2 = df2.astype({'시간':'int16'})
df2 = df2.astype({'시가':'int32'})
df2 = df2.astype({'고가':'int32'})
df2 = df2.astype({'저가':'int32'})
df2 = df2.astype({'종가':'int32'})
df2 = df2.astype({'전일대비':'int32'})
df2 = df2.astype({'거래량':'int32'})
df2 = df2.astype({'거래대금':'int64'})
df2 = df2.astype({'누적체결매도수량':'int32'})
df2 = df2.astype({'상장주식수':'int32'})
df2 = df2.astype({'시가총액':'int64'})
df2 = df2.astype({'외국인주문한도수량':'int32'})
df2 = df2.astype({'외국인주문가능수량':'int32'})
df2 = df2.astype({'외국인현보유수량':'int32'})
df2 = df2.astype({'외국인현보유비율':'float32'})
df2 = df2.astype({'수정주가일자':'int32'})
df2 = df2.astype({'수정주가비율':'float32'})
df2 = df2.astype({'기관순매수량':'int32'})
df2 = df2.astype({'기관누적순매수량':'int32'})
df2 = df2.astype({'등락주선':'int32'})
df2 = df2.astype({'등락비율':'float32'})
df2 = df2.astype({'예탁금':'int64'})
df2 = df2.astype({'주식회전율':'float32'})
df2 = df2.astype({'거래성립률':'float32'})


In [ ]:
sql = 'SELECT COUNT(*) FROM STOCK'
print(pd.read_sql(sql=sql, con=db1))

   COUNT(*)
0    190380


c:\Users\TJ\.conda\envs\tf2\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
